## This is used to generate runtime plots presented in the appendix.

This uses the output generated by *_analyzer.py to get the number of evaluations needed
 to reach a certain performance threshold.

The algorithm runtimes are separately calculated and manually input here.

In [117]:
from typing import List
import torch
%matplotlib
import matplotlib.pyplot as plt

bw_cvar_data = torch.load("bw_cvar_analyzer_output.pt")
marzat_data = torch.load("marzat_analyzer_output.pt")

# this lists the algorithms we want to include along with the corresponding runtime per
# iteration measured in seconds.
keys = [
    "EI",
    "MES",
    "qKG",
    "UCB",
    "classical_random",
    "random",
    "tts_apx_q=1",
    # "apx_cvar_q=1",
    "tts_rhoKG_q=1",
]

# color list [u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd',
# u'#8c564b', u'#e377c2', u'#7f7f7f', u'#bcbd22', u'#17becf']


# the plot parameters for each key
params_dict = {
    "EI": {"label": "EI", "marker": "x", "linestyle": "dotted", "errorevery": 1,
           "color": u'#1f77b4'},
    "MES": {"label": "MES", "marker": "x", "linestyle": "dashed", "errorevery": 1,
            "color": u'#ff7f0e'},
    "qKG": {"label": "KG", "marker": "x", "linestyle": "dashdot", "errorevery": 1,
            "color": u'#2ca02c'},
    "UCB": {"label": "UCB", "marker": "x", "linestyle": (0, (5, 10)), "errorevery": 1,
            "color": u'#d62728'},
    "classical_random": {
        "label": "random",
        "marker": "x",
        "linestyle": (0, (3, 10, 1, 10)),
        "errorevery": 1,
        "color": u'#9467bd'
    },
    "random": {"label": "$\\rho$-random", "marker": "p", "errorevery": 5, "color":
        u'#8c564b'},
    "tts_apx_q=1": {"label": "$\\rho$KG$^{apx}$", "marker": "+", "errorevery": 5,
                    "color": u'#e377c2'},
    "tts_rhoKG_q=1": {"label": "$\\rho$KG", "marker": "s", "errorevery": 5, "color":
        u'#7f7f7f'},
}

for key, value in params_dict.items():
    value.pop("errorevery")

# number of evaluations per iteration by benchmarks
bw_multiplier = 12
marzat_multiplier = 8

# bw runtimes are measured using a GTX 1660 GPU
bw_runtime = {
    "tts_apx_q=1": 45.,
    "apx_cvar_q=1": 45.,
    "tts_rhoKG_q=1": 190.,
    "random": 0.5,
    "classical_random": 0.1,
    "UCB": 0.25,
    "qKG": 3.,
    "MES": 1.,
    "EI": 1.,
}

# marzat runtimes are measured using AWS c5n.2xlarge
marzat_runtime = {
    "tts_apx_q=1": 215.,
    "apx_cvar_q=1": 44.,
    "tts_rhoKG_q=1": 4250.,
    "random": 1.0,
    "classical_random": 0.1,
    "UCB": 0.5,
    "qKG": 6.,
    "MES": 1.5,
    "EI": 1.,
}

# the threshold log gap
bw_threshold = 320  # 300 is approximately log 2.477
marzat_threshold = 1  # approximately log -0.3


Using matplotlib backend: Qt5Agg


In [100]:
# calculate the number of iterations it takes to get to the threshold
# 999 means it was never crossed

bw_iterations_needed = dict()
marzat_iterations_needed = dict()

for prob in ["bw", "marzat"]:
    if prob == "bw":
        iterations_needed = bw_iterations_needed
        data = bw_cvar_data
        threshold = bw_threshold
    else:
        iterations_needed = marzat_iterations_needed
        data = marzat_data
        threshold = marzat_threshold
    for key in keys:
        key_data = data.get(key, None)
        if key_data is None:
            iterations_needed[key] = 999
        else:
            avg = key_data["y"].mean(dim=0)
            below_threshold = avg < threshold
            # count as crossed threshold if two consecutive values cross
            try:
                for i in range(int(below_threshold.shape[0])):
                    if below_threshold[i] * below_threshold[i+1]:
                        iterations_needed[key] = i
                        break
            except IndexError:
                iterations_needed[key] = 999


In [101]:
def time_needed(prob: str, evaluation_time: List[float]):
    r"""
    Converts iterations needed to total runtime
    :param prob: The problem to use
    :param evaluation_time: List of function evaluation time in seconds
    :return: Dictionary of corresponding total runtimes to reach the threshold
                This dictionary ignores keys with iterations_needec = 999
    """
    evaluation_time = torch.tensor(evaluation_time)
    output = dict()
    if prob == "bw":
        iterations_needed = bw_iterations_needed
        runtime = bw_runtime
        multiplier = bw_multiplier
    else:
        iterations_needed = marzat_iterations_needed
        runtime = marzat_runtime
        multiplier = marzat_multiplier
    for i, key in enumerate(keys):
        if iterations_needed[key] == 999:
            continue
        active_multiplier = multiplier if i < 5 else 1
        output[key] = (runtime[key] + evaluation_time * active_multiplier) * iterations_needed[key]
    return output

In [125]:
# Plot for BW
times = [0, 60, 120, 300, 600, 1200, 1800]  # in seconds
plot_data = time_needed("bw", times)

plt.figure()
plt.title("Branin Williams CVaR - Time to reach the optimality gap of %d" %
          bw_threshold)
plt.xlabel("Function evaluation time in minutes")
plt.ylabel("Total time in minutes")
for key, data in plot_data.items():
    plt.plot(torch.tensor(times)/60., data/60., **params_dict[key])
plt.yscale("log")
plt.grid(True)
plt.legend()
plt.savefig("bw_time_to_benchmark.png", dpi=300)
plt.show()

In [126]:
# Plot for marzat
plot_data = time_needed("marzat", times)

plt.figure()
plt.title("$f_6(x_c, x_e)$ - Time to reach the optimality gap of %.1f" %
          marzat_threshold)
plt.xlabel("Function evaluation time in minutes")
plt.ylabel("Total time in minutes")
for key, data in plot_data.items():
    plt.plot(torch.tensor(times)/60., data/60., **params_dict[key])
plt.yscale("log")
plt.grid(True)
plt.legend()
plt.savefig("marzat_time_to_benchmark.png", dpi=300)
plt.show()


### From here on, we will produce alternative plots, which are just the same plots that are presented in the main paper, except that the x axis is replaced with the algorithm runtimes.


In [124]:
evaluation_time = 300.  # in seconds

# TODO: add error bars here!!

# Plot for BW
data = bw_cvar_data
runtime = bw_runtime
plt.figure()
plt.title("Branin Williams log optimality gap, with evaluation time of %.1d mins" %
          (evaluation_time/60.))
plt.xlabel("Total runtime in minutes")
plt.ylabel("Log optimality gap")
for i, key in enumerate(keys):
    if data.get(key) is None:
        continue
    avg_log_gap = torch.mean(torch.log10(data[key]["y"]), dim=0)
    std_log_gap = torch.std(torch.log10(data[key]["y"]), dim=0) / torch.sqrt(
        torch.tensor(data[key]["y"].size(0), dtype=torch.float)
    )
    y = torch.log10(data[key]["y"]).mean(dim=0)
    modified_x = data[key]["x"] * evaluation_time + runtime[key] * torch.arange\
        (start=1, end=data[key]["x"].shape[0] + 1)
    markers, caps, bars = plt.errorbar(
        modified_x/60., avg_log_gap, yerr=std_log_gap.cpu(), **params_dict[key], ms=5,
                errorevery=1 if i < 5 else 5
    )
    [bar.set_alpha(0.5) for bar in bars]
    # plt.plot(modified_x/60., y, **params_dict[key])
plt.grid(True)
plt.legend()
plt.xlim((0, 240*evaluation_time/60.))
plt.show()


# Plot for marzat
data = marzat_data
runtime = marzat_runtime
plt.figure()
plt.title("$f_6(x_c, x_e)$ log optimality gap, with evaluation time of %d "
          "mins" % (evaluation_time/60.))
plt.xlabel("Total runtime in minutes")
plt.ylabel("Log optimality gap")
for i, key in enumerate(keys):
    if data.get(key) is None:
        continue
    avg_log_gap = torch.mean(torch.log10(data[key]["y"]), dim=0)
    std_log_gap = torch.std(torch.log10(data[key]["y"]), dim=0) / torch.sqrt(
        torch.tensor(data[key]["y"].size(0), dtype=torch.float)
    )
    y = torch.log10(data[key]["y"]).mean(dim=0)
    modified_x = data[key]["x"] * evaluation_time + runtime[key] * torch.arange\
        (start=1, end=data[key]["x"].shape[0] + 1)
    markers, caps, bars = plt.errorbar(
        modified_x/60., avg_log_gap, yerr=std_log_gap.cpu(), **params_dict[key], ms=5,
        errorevery=1 if i < 5 else 5
    )
    [bar.set_alpha(0.5) for bar in bars]
plt.grid(True)
plt.xlim((0, 200*evaluation_time/60.))
plt.legend()
plt.show()